In [12]:
from dateutil import rrule
import urllib.request
import datetime
import pandas as pd
import numpy as np
import json
import time
import ssl
import datetime
from collections import defaultdict

In [13]:
stock = 2330
days = 60
start_from_day_ago = 214
current_day = 1
stock_balance = 0
init_assets = 2000000 # should be greater than 0

In [14]:
# initialize the Indices
RSI = 0
KD = 0
MACD =0
Bollinger_Band = 0
D_value = 0
J_value = 0

In [15]:
fields = None
data_record = None
open_day = []
for i in range(1, 13):
  # Get the all data
  url = (
    "https://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="
    # + datetime.datetime.now().strftime("%Y%m%d")
    + f"2023{i:02d}01"
    + "&stockNo="
    + str(stock)
  )
  headers={"User-Agent":"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE"}

  req = urllib.request.Request(url, headers=headers)
  req_data = json.loads(urllib.request.urlopen(req).read())
  if i == 1:
    fields = req_data['fields']
    data_record = req_data['data']
  else:
    for j in range(0, len(req_data['data'])):
      data_record.append(req_data['data'][j])
  open_day.append(len(req_data['data']))

In [16]:
df = pd.DataFrame(data_record,columns=fields)
df

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,112/01/03,"15,311,364","6,871,973,708",446.00,453.50,443.00,453.00,+4.50,"22,581"
1,112/01/04,"20,626,874","9,310,050,329",449.50,455.00,448.50,449.50,-3.50,"18,233"
2,112/01/05,"23,972,099","10,972,616,269",459.00,459.50,455.00,458.50,+9.00,"20,752"
3,112/01/06,"21,313,593","9,745,142,549",455.00,459.50,455.00,458.50,0.00,"16,635"
4,112/01/09,"49,186,355","23,352,375,299",468.00,481.00,467.50,481.00,+22.50,"57,305"
...,...,...,...,...,...,...,...,...,...
234,112/12/25,"9,994,163","5,820,235,840",582.00,585.00,580.00,581.00,-1.00,"12,786"
235,112/12/26,"17,033,926","9,963,615,754",583.00,586.00,582.00,586.00,+5.00,"22,883"
236,112/12/27,"34,750,958","20,491,144,772",587.00,592.00,586.00,592.00,+6.00,"37,372"
237,112/12/28,"27,152,352","16,064,106,365",592.00,593.00,589.00,593.00,+1.00,"26,497"


In [17]:
open_day

[13, 18, 23, 17, 22, 20, 21, 22, 20, 20, 22, 21]

In [18]:
df.set_index('日期',inplace=True)
pd.set_option('display.max_columns',1000)
pd.set_option('display.width',1000)
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.unicode.ambiguous_as_wide',True)
pd.set_option('display.unicode.east_asian_width',True)
df['收盤價']=df['收盤價'].astype(float)
df['開盤價']=df['開盤價'].astype(float)
df['最低價']=df['最低價'].astype(float)
df['最高價']=df['最高價'].astype(float)

In [19]:
# RSI
close_price_row = df['收盤價']

# create a new data frame for analyze
analyze_df = pd.DataFrame()

# 每日漲跌幅度
analyze_df['漲跌'] = df['收盤價'].diff()
# 14天內的平均漲幅和跌幅
l = 14
analyze_df['平均UP'] = analyze_df['漲跌'].apply(lambda x: x if x > 0 else 0).rolling(l).mean()
analyze_df['平均DN'] = analyze_df['漲跌'].apply(lambda x: -x if x < 0 else 0).rolling(l).mean()
# 相對強弱（RS）
analyze_df['RS'] = analyze_df['平均UP'] / analyze_df['平均DN']
# RSI
analyze_df['RSI'] = 100 - (100 / (1 + analyze_df['RS']))
analyze_df

,漲跌,平均UP,平均DN,RS,RSI
日期,,,,,
112/01/03,NaN,NaN,NaN,NaN,NaN
112/01/04,-3.5,NaN,NaN,NaN,NaN
112/01/05,9.0,NaN,NaN,NaN,NaN
112/01/06,0.0,NaN,NaN,NaN,NaN
112/01/09,22.5,NaN,NaN,NaN,NaN
...,...,...,...,...,...
112/12/25,-1.0,1.785714,1.000000,1.785714,64.102564
112/12/26,5.0,2.142857,1.000000,2.142857,68.181818
112/12/27,6.0,2.571429,0.714286,3.600000,78.260870


In [20]:
 #boll

# 中軌（20日均線）
analyze_df['中軌'] = df['收盤價'].rolling(window=20).mean()

# 標準差
std = df['收盤價'].rolling(window=20).std()

analyze_df['上軌'] = analyze_df['中軌'] + 2 * std
analyze_df['下軌'] = analyze_df['中軌'] - 2 * std
analyze_df['帶寬'] = (analyze_df['上軌'] - analyze_df['下軌']) / analyze_df['中軌']


# for i, df_row in df.iterrows():
#     date = df_row['日期']
#     open_price = df_row['開盤價']
#     high = df_row['最高價']
#     low = df_row['最低價']
#     close = df_row['收盤價']

In [21]:
analyze_df

,漲跌,平均UP,平均DN,RS,RSI,中軌,上軌,下軌,帶寬
日期,,,,,,,,,
112/01/03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112/01/04,-3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112/01/05,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112/01/06,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
112/01/09,22.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
112/12/25,-1.0,1.785714,1.000000,1.785714,64.102564,577.30,588.763443,565.836557,0.039714
112/12/26,5.0,2.142857,1.000000,2.142857,68.181818,577.85,589.889846,565.810154,0.041671
112/12/27,6.0,2.571429,0.714286,3.600000,78.260870,578.75,592.187967,565.312033,0.046438
